<a href="https://colab.research.google.com/github/fralfaro/MAT306/blob/main/docs/labs/lab_04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MAT306 - Laboratorio N°04


**Objetivo**: Aplicar técnicas intermedias y avanzadas de análisis de datos con pandas utilizando un caso real: el Índice de Libertad de Prensa. Este laboratorio incluye operaciones de limpieza, transformación, combinación de datos, y análisis exploratorio usando `merge`, `groupby`, `concat` y otras funciones fundamentales.




**Descripción del Dataset**

El presente conjunto de datos está orientado al análisis del **Índice de Libertad de Prensa**, una métrica internacional que evalúa el nivel de libertad del que gozan periodistas y medios de comunicación en distintos países. Este índice es recopilado anualmente por la organización **Reporteros sin Fronteras**.

La base de datos contempla observaciones por país y año, e incluye tanto el valor del índice como el ranking correspondiente. A menor puntaje en el índice, mayor nivel de libertad de prensa.

**Diccionario de variables**

| Variable     | Clase    | Descripción                                                                          |
| ------------ | -------- | ------------------------------------------------------------------------------------ |
| `codigo_iso` | carácter | Código ISO 3166-1 alfa-3 que representa a cada país.                                 |
| `pais`       | carácter | Nombre oficial del país.                                                             |
| `anio`       | entero   | Año en que se registró la medición del índice.                                       |
| `indice`     | numérico | Valor numérico del Índice de Libertad de Prensa (menor valor indica mayor libertad). |
| `ranking`    | entero   | Posición relativa del país en el ranking mundial de libertad de prensa.              |


**Fuente original y adaptación pedagógica**

* **Fuente original**: [Reporteros sin Fronteras](https://www.rsf-es.org/), recopilado y publicado a través del portal del [Banco Mundial](https://tcdata360.worldbank.org/indicators/h3f86901f?country=BRA&indicator=32416&viz=line_chart&years=2001,2019).
* **Adaptación educativa**: Los archivos han sido modificados intencionalmente para incorporar desafíos técnicos que permiten aplicar los contenidos abordados en clases, tales como limpieza de datos, normalización, detección de duplicados, y combinación de fuentes.


**Descripción de los archivos disponibles**

* **`libertad_prensa_codigo.csv`**: Contiene los pares `codigo_iso` y `pais`. Incluye intencionalmente un código ISO con dos nombres distintos de país para efectos de limpieza y validación de datos.

* **`libertad_prensa_01.csv`**: Contiene registros de los años **anteriores a 2010**. Incluye las variables `PAIS`, `ANIO`, `INDICE`, y `RANKING` con nombres de columna en **mayúsculas**.

* **`libertad_prensa_02.csv`**: Contiene registros de los años **desde 2010 en adelante**. Estructura similar al archivo anterior, con nombres de columna también en **mayúsculas**.





In [1]:
import numpy as np
import pandas as pd

# lectura de datos
archivos_anio = [
    'https://raw.githubusercontent.com/fralfaro/MAT306/main/docs/labs/data/libertad_prensa_01.csv',
    'https://raw.githubusercontent.com/fralfaro/MAT306/main/docs/labs/data/libertad_prensa_02.csv'
 ]
df_codigos = pd.read_csv('https://raw.githubusercontent.com/fralfaro/MAT306/main/docs/labs/data/libertad_prensa_codigo.csv')



### 1. Consolidación y limpieza de datos

A partir de los archivos disponibles, realice los siguientes pasos:

**a)** Cree un DataFrame llamado `df_anio` que consolide la información proveniente de los archivos **`libertad_prensa_01.csv`** y **`libertad_prensa_02.csv`**, correspondientes a distintas ventanas de tiempo. Recuerde que ambos archivos tienen nombres de columnas en mayúscula, por lo que debe normalizarlas a **minúscula** para asegurar consistencia.

**b)** Explore el archivo **`libertad_prensa_codigo.csv`** e identifique el código ISO que aparece asociado a dos nombres de país distintos. Elimine el registro que corresponda a un valor incorrecto o inconsistente, conservando solo el que considere válido.

**c)** Una vez preparados los archivos, cree un nuevo DataFrame llamado `df` que combine `df_anio` con `df_codigos`, utilizando la columna `codigo_iso` como clave. Asegúrese de realizar una unión que conserve únicamente los registros que tengan coincidencia en ambas fuentes.

> **Sugerencia**:
>
> * Para unir los archivos por filas (años), utilice la función `pd.concat([...])`.
> * Para combinar información por columnas (variables), utilice `pd.merge(...)` especificando `on='codigo_iso'`.



In [34]:

#a)
df_anio = pd.concat([pd.read_csv(a) for a in archivos_anio], ignore_index=True)

df_anio.columns = df_anio.columns.str.lower()

print(df_anio.head())


#b)
duplicados = df_codigos[df_codigos.duplicated('codigo_iso', keep=False)]
print(duplicados)
df_codigos = df_codigos[df_codigos['pais'] != 'PaísFalso']


#c)

df = pd.merge(df_anio, df_codigos, on='codigo_iso', how='inner')

print(df.head())


  codigo_iso  anio  indice  ranking
0        AFG  2001    35.5     59.0
1        AGO  2001    30.2     50.0
2        ALB  2001     NaN      NaN
3        AND  2001     NaN      NaN
4        ARE  2001     NaN      NaN
    codigo_iso      pais
179        ZWE  Zimbabue
180        ZWE      malo
  codigo_iso  anio  indice  ranking                    pais
0        AFG  2001    35.5     59.0             Afghanistán
1        AGO  2001    30.2     50.0                  Angola
2        ALB  2001     NaN      NaN                 Albania
3        AND  2001     NaN      NaN                 Andorra
4        ARE  2001     NaN      NaN  Emiratos Árabes Unidos




### 2. Exploración inicial del conjunto de datos

Una vez que hayas consolidado el DataFrame final `df`, realiza un análisis exploratorio básico respondiendo las siguientes preguntas:

#### **Estructura del DataFrame**

* ¿Cuántas **filas (observaciones)** contiene el conjunto de datos?
* ¿Cuántas **columnas** tiene el DataFrame?
* ¿Cuáles son los **nombres de las columnas**?
* ¿Qué **tipo de datos** tiene cada columna?
* ¿Hay columnas con un tipo de dato inesperado (por ejemplo, fechas como strings)?

#### **Resumen estadístico**

* Genera un resumen estadístico del conjunto de datos con `.describe()`.
  ¿Qué observas sobre los valores de `indice` y `ranking`?
* ¿Qué valores mínimo, máximo y promedio tiene la columna `indice`?
* ¿Qué países presentan los valores extremos en `indice` y `ranking`?

#### **Datos faltantes**

* ¿Cuántos valores nulos hay en cada columna?
* ¿Qué proporción de observaciones tienen valores faltantes?
* ¿Hay columnas con más del 30% de datos faltantes?

#### **Unicidad y duplicados**

* ¿Cuántos países distintos (`pais`) hay en el DataFrame?
* ¿Cuántos años distintos (`anio`) hay representados?
* ¿Existen filas duplicadas (exactamente iguales)? ¿Cuántas?

#### **Validación cruzada de columnas**

* ¿Hay inconsistencias entre el país (`pais`) y su código (`codigo_iso`)?
  (por ejemplo, un mismo código ISO asociado a más de un país)

> **Sugerencia**: Apoya tu análisis con funciones como `.info()`, `.nunique()`, `.isnull().sum()`, `.duplicated()`, `.value_counts()`, entre otras.



    

In [29]:
print("Número de filas:", df.shape[0])
print("Número de columnas:", df.shape[1])
print("\nNombres de las columnas:", df.columns.tolist())
print("\nTipos de datos por columna:")
print(df.dtypes)


print("\nResumen estadístico:")
print(df.describe())

print("\nIndice - Min:", df['indice'].min(),
      " Max:", df['indice'].max(),
      " Promedio:", df['indice'].mean())

pais_min_indice = df.loc[df['indice'].idxmin(), ['pais', 'anio', 'indice']]
pais_max_indice = df.loc[df['indice'].idxmax(), ['pais', 'anio', 'indice']]
pais_mejor_ranking = df.loc[df['ranking'].idxmin(), ['pais', 'anio', 'ranking']]
pais_peor_ranking = df.loc[df['ranking'].idxmax(), ['pais', 'anio', 'ranking']]

print("\nPaís con menor índice (mejor libertad):\n", pais_min_indice)
print("\nPaís con mayor índice (peor libertad):\n", pais_max_indice)
print("\nPaís con mejor ranking:\n", pais_mejor_ranking)
print("\nPaís con peor ranking:\n", pais_peor_ranking)

print("\nValores nulos por columna:")
print(df.isnull().sum())

print("\nProporción de observaciones con valores faltantes:",
      (df.isnull().any(axis=1).mean() * 100).round(2), "%")

print("\nColumnas con más del 30% de valores nulos:")
print(df.isnull().mean()[df.isnull().mean() > 0.3])


print("\nNúmero de países distintos:", df['pais'].nunique())
print("Número de años distintos:", df['anio'].nunique())
print("Número de filas duplicadas:", df.duplicated().sum())


print("\nRevisión de inconsistencias (códigos asociados a >1 país):")
print(df.groupby('codigo_iso')['pais'].nunique().value_counts())
print("\nCódigos conflictivos:")
print(df.groupby('codigo_iso')['pais'].nunique()[df.groupby('codigo_iso')['pais'].nunique() > 1])

Número de filas: 3077
Número de columnas: 5

Nombres de las columnas: ['codigo_iso', 'anio', 'indice', 'ranking', 'pais']

Tipos de datos por columna:
codigo_iso     object
anio            int64
indice        float64
ranking       float64
pais           object
dtype: object

Resumen estadístico:
              anio        indice        ranking
count  3077.000000   2680.000000    2854.000000
mean   2009.941176    204.836847     475.715137
std       5.786018   2687.491660    6455.679267
min    2001.000000      0.000000       1.000000
25%    2005.000000     15.490000      35.000000
50%    2009.000000     28.150000      71.000000
75%    2015.000000     41.447500     110.000000
max    2019.000000  64536.000000  121056.000000

Indice - Min: 0.0  Max: 64536.0  Promedio: 204.83684701492538

País con menor índice (mejor libertad):
 pais      Dinamarca
anio           2008
indice          0.0
Name: 1311, dtype: object

País con mayor índice (peor libertad):
 pais       Kosovo
anio         2014
ind




### 3. Comparación regional: países latinoamericanos

En esta sección se busca identificar cuáles son los países de América Latina que han presentado los valores extremos del **Índice de Libertad de Prensa** en cada año observado.

> Recuerda que un menor puntaje en `indice` implica mayor libertad de prensa.

#### **Tareas:**

**a)** Utilizando un ciclo `for`, recorre cada año del conjunto de datos filtrado por países latinoamericanos, y determina para cada año:

* El país con el menor valor de `indice` (mayor libertad de prensa).
* El país con el mayor valor de `indice` (menor libertad de prensa).

**b)** Resuelve la misma tarea del punto anterior utilizando un enfoque vectorizado con `groupby`, sin usar ciclos explícitos.



#### **Lista de países latinoamericanos considerada:**

```python
america = ['ARG', 'ATG', 'BLZ', 'BOL', 'BRA', 'CAN', 'CHL', 'COL', 'CRI',
           'CUB', 'DOM', 'ECU', 'GRD', 'GTM', 'GUY', 'HND', 'HTI', 'JAM',
           'MEX', 'NIC', 'PAN', 'PER', 'PRY', 'SLV', 'SUR', 'TTO', 'URY',
           'USA', 'VEN']
```

> Puedes usar esta lista para filtrar el DataFrame final por la columna `codigo_iso`.



In [33]:
# respuesta
america = ['ARG', 'ATG', 'BLZ', 'BOL', 'BRA', 'CAN', 'CHL', 'COL', 'CRI',
       'CUB', 'DOM', 'ECU', 'GRD', 'GTM', 'GUY', 'HND', 'HTI', 'JAM',
       'MEX', 'NIC', 'PAN', 'PER', 'PRY', 'SLV', 'SUR', 'TTO', 'URY',
       'USA', 'VEN']

# Filtrar solo países de América
df_america = df[df['codigo_iso'].isin(america)]

for anio in sorted(df_america['anio'].unique()):
    datos_anio = df_america[df_america['anio'] == anio]

    # Saltar año si no hay datos
    if datos_anio.empty or datos_anio['indice'].isna().all():
        print(f"Año {anio}: no hay datos disponibles")
        continue

    pais_min = datos_anio.loc[datos_anio['indice'].idxmin(), ['pais','indice']]
    pais_max = datos_anio.loc[datos_anio['indice'].idxmax(), ['pais','indice']]

    print(f"Año {anio}:")
    print(f" - Mayor libertad (menor índice): {pais_min['pais']} ({pais_min['indice']})")
    print(f" - Menor libertad (mayor índice): {pais_max['pais']} ({pais_max['indice']})")
    print("-"*50)


df_america_valido = df_america.dropna(subset=['indice'])

# País con menor índice por año
minimos = df_america_valido.loc[df_america_valido.groupby("anio")["indice"].idxmin(), ["anio","pais","indice"]]
minimos = minimos.rename(columns={"pais":"pais_mejor","indice":"indice_min"})

# País con mayor índice por año
maximos = df_america_valido.loc[df_america_valido.groupby("anio")["indice"].idxmax(), ["anio","pais","indice"]]
maximos = maximos.rename(columns={"pais":"pais_peor","indice":"indice_max"})

extremos = pd.merge(minimos, maximos, on="anio")
print(extremos)

Año 2001:
 - Mayor libertad (menor índice): Canadá (0.8)
 - Menor libertad (mayor índice): Cuba (90.3)
--------------------------------------------------
Año 2002:
 - Mayor libertad (menor índice): Trinidad y Tobago (1.0)
 - Menor libertad (mayor índice): Cuba (97.83)
--------------------------------------------------
Año 2003:
 - Mayor libertad (menor índice): Trinidad y Tobago (2.0)
 - Menor libertad (mayor índice): Argentina (35826.0)
--------------------------------------------------
Año 2004:
 - Mayor libertad (menor índice): Trinidad y Tobago (2.0)
 - Menor libertad (mayor índice): Cuba (87.0)
--------------------------------------------------
Año 2005:
 - Mayor libertad (menor índice): Bolivia (4.5)
 - Menor libertad (mayor índice): Cuba (95.0)
--------------------------------------------------
Año 2006:
 - Mayor libertad (menor índice): Canadá (4.88)
 - Menor libertad (mayor índice): Cuba (96.17)
--------------------------------------------------
Año 2007:
 - Mayor libertad (me

### 4. Análisis anual del índice por país

En esta sección se busca analizar la evolución del **índice máximo** de libertad de prensa alcanzado por cada país a lo largo del tiempo.

#### **Tarea principal:**

* Construye una tabla dinámica (`pivot_table`) donde las **filas** correspondan a los países, las **columnas** a los años (`anio`) y los **valores** sean el `indice` máximo alcanzado por cada país en ese año.
* Asegúrate de reemplazar los valores nulos resultantes con `0`.

> **Hint**: Puedes utilizar el parámetro `fill_value=0` en `pd.pivot_table(...)`.



#### **Preguntas adicionales:**

**a)** ¿Qué país tiene el mayor valor de `indice` en toda la tabla resultante? ¿Y cuál tiene el menor (distinto de cero)?
**b)** ¿Qué años presentan en promedio los valores de `indice` más altos? ¿Y los más bajos?

> (Pista: usa `.mean(axis=0)` sobre la tabla pivot)

**c)** ¿Qué país muestra mayor **variabilidad** (diferencia entre su máximo y mínimo `indice` a lo largo del tiempo)?

> (Pista: aplica `.max(axis=1) - .min(axis=1)`)

**d)** ¿Existen países con índice constante a lo largo de todos los años registrados? ¿Cuáles?

**e)** ¿Qué países no tienen ningún dato (es decir, quedaron con todos los valores igual a 0)? ¿Podrías explicar por qué?





In [42]:
tabla_indice_max = pd.pivot_table(
    df,
    index='pais',
    columns='anio',
    values='indice',
    aggfunc='max',
    fill_value=0
)

print(tabla_indice_max.head())

#a)

mayor_valor = tabla_indice_max.max().max()
pais_mayor = tabla_indice_max.stack().idxmax()

tabla_sin_ceros = tabla_indice_max.replace(0, np.nan)
menor_valor = tabla_sin_ceros.min().min()
pais_menor = tabla_sin_ceros.stack().idxmin()

print(f"Mayor índice: {mayor_valor} en {pais_mayor}")
print(f"Menor índice (distinto de 0): {menor_valor} en {pais_menor}")

#b)
promedio_por_anio = tabla_indice_max.mean(axis=0)

anio_max_prom = promedio_por_anio.idxmax()
anio_min_prom = promedio_por_anio.idxmin()

print(f"Año con promedio más alto: {anio_max_prom} ({promedio_por_anio[anio_max_prom]:.2f})")
print(f"Año con promedio más bajo: {anio_min_prom} ({promedio_por_anio[anio_min_prom]:.2f})")

#c)
variabilidad = tabla_indice_max.max(axis=1) - tabla_indice_max.min(axis=1)
pais_mayor_var = variabilidad.idxmax()
print(f"País con mayor variabilidad de índice: {pais_mayor_var} ({variabilidad[pais_mayor_var]:.2f})")

#d)
constantes = tabla_indice_max[(tabla_indice_max.max(axis=1) - tabla_indice_max.min(axis=1)) == 0]
print("Países con índice constante:", constantes.index.tolist())
#Notamos que ninguno.

#e)
sin_datos = tabla_indice_max[(tabla_indice_max == 0).all(axis=1)]
print("Países sin ningún dato:", sin_datos.index.tolist())
#Nuevamente ninguno.


anio         2001   2002   2003   2004   2005   2006   2007   2008   2009  \
pais                                                                        
Afghanistán  35.5  40.17  28.25  39.17  44.25  56.50  59.25  54.25  51.67   
Albania       0.0   6.50  11.50  14.17  18.00  25.50  16.00  21.75  21.50   
Alemania      1.5   1.33   2.00   4.00   5.50   5.75   4.50   3.50   4.25   
Algeria      31.0  33.00  43.50  40.33  40.00  40.50  31.33  49.56  47.33   
Andorra       0.0   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   

anio          2012   2013   2014   2015   2017   2018   2019  
pais                                                          
Afghanistán  37.36  37.07  37.44  37.75  39.46  37.28  36.55  
Albania      30.88  29.92  28.77  29.92  29.92  29.49  29.84  
Alemania     10.24  10.23  11.47  14.80  14.97  14.39  14.60  
Algeria      36.54  36.26  36.63  41.69  42.83  43.13  45.75  
Andorra       6.82   6.82  19.87  19.87  21.03  22.21  24.63  
Mayor índice: 64536